In [1]:

import numpy as np
import pandas as pd

matrix = np.array([
    [0.191, 0.081, 0.067, 0.111],
    [0.082, 0.059, 0.063, 0.102],
    [0.036, 0.083, 0.076, 0.088],
    [0.029, 0.088, 0.235, 0.078],
    [0.306, 0.055, 0.113, 0.068],
    [0.038, 0.154, 0.033, 0.144],
    [0.014, 0.100, 0.047, 0.148],
    [0.029, 0.155, 0.139, 0.105],
    [0.014, 0.083, 0.150, 0.089], 
    [0.010, 0.142, 0.067, 0.069]
])

criteria = ['C1', 'C2', 'C3', 'C4']
sites = ['Site1', 'Site2', 'Site3', 'Site4', 'Site5', 'Site6', 'Site7', 'Site8', 'Site9', 'Site10']
criteria_types = ['max', 'max', 'max', 'max']
weights = np.array([0.25377794, 0.35076977, 0.39287723, 0.00257506])

# V-shape preference thresholds (can be tuned)
p_list = [2.998, 1.441, 0.783, 3.957]

def normalize_matrix(matrix, criteria_types):
    norm_matrix = np.zeros_like(matrix, dtype=float)
    for j in range(matrix.shape[1]):
        col = matrix[:, j]
        if criteria_types[j] == 'max':
            norm_matrix[:, j] = (col - np.min(col)) / (np.max(col) - np.min(col))
        else:  # 'min'
            norm_matrix[:, j] = (np.max(col) - col) / (np.max(col) - np.min(col))
    return norm_matrix

def vshape_preference(d, p):
    return np.clip(d / p, 0, 1)

def calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list):
    n = norm_matrix.shape[0]
    m = norm_matrix.shape[1]
    preference_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            if i == j:
                continue
            pref_sum = 0
            for k in range(m):
                if criteria_types[k] == 'max':
                    diff = norm_matrix[i, k] - norm_matrix[j, k]
                else:  # 'min'
                    diff = norm_matrix[j, k] - norm_matrix[i, k]
                pref = vshape_preference(diff, p_list[k])
                pref_sum += weights[k] * pref
            preference_matrix[i, j] = pref_sum
    return preference_matrix

def calculate_flows(preference_matrix):
    n = preference_matrix.shape[0]
    phi_plus = preference_matrix.sum(axis=1) / (n - 1)
    phi_minus = preference_matrix.sum(axis=0) / (n - 1)
    net_flows = phi_plus - phi_minus
    return phi_plus, phi_minus, net_flows

def promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria):
    norm_matrix = normalize_matrix(matrix, criteria_types)
    preference_matrix = calculate_preference_matrix(norm_matrix, criteria_types, weights, p_list)
    phi_plus, phi_minus, net_flows = calculate_flows(preference_matrix)
    
    # Prepare results DataFrame
    df = pd.DataFrame(matrix, columns=criteria, index=sites)
    df['Leaving Flow'] = phi_plus
    df['Entering Flow'] = phi_minus
    df['Net Flow'] = net_flows
    df['Rank'] = df['Net Flow'].rank(ascending=False, method='min').astype(int)
    df = df.sort_values('Rank')
    return df

# Run PROMETHEE II
result_df = promethee_ii(matrix, criteria_types, weights, p_list, sites, criteria)
print(result_df)


           C1     C2     C3     C4  Leaving Flow  Entering Flow  Net Flow  \
Site4   0.029  0.088  0.235  0.078      0.368486       0.070183  0.298303   
Site8   0.029  0.155  0.139  0.105      0.290367       0.045753  0.244614   
Site9   0.014  0.083  0.150  0.089      0.178997       0.103672  0.075326   
Site10  0.010  0.142  0.067  0.069      0.136367       0.129279  0.007088   
Site5   0.306  0.055  0.113  0.068      0.163131       0.173061 -0.009930   
Site6   0.038  0.154  0.033  0.144      0.149773       0.185108 -0.035335   
Site1   0.191  0.081  0.067  0.111      0.069696       0.169701 -0.100005   
Site3   0.036  0.083  0.076  0.088      0.045746       0.167445 -0.121699   
Site7   0.014  0.100  0.047  0.148      0.045264       0.199466 -0.154202   
Site2   0.082  0.059  0.063  0.102      0.026721       0.230881 -0.204160   

        Rank  
Site4      1  
Site8      2  
Site9      3  
Site10     4  
Site5      5  
Site6      6  
Site1      7  
Site3      8  
Site7      9  
Si